# Algoritmo para separar os dados do movimento desejado, utilizando os dados do vídeo com os dados de MoCap que foram sincronizados

--> Bibliotecas necessárias para esta etapa

In [1]:
import os
import collections
import re
import cv2

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal as signal
from pathlib import Path
from sources_cv2 import defineStartEnd, count_frames

from utils import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Caminho onde estão os dados .mat
caminho = r'C:\\Users\\gusta\\OneDrive\\Documentos\\Users\\Gustavo Monteiro\\Documents\\GitHub\\MoVi-Toolbox\\MoCap\\'

# Escolha do voluntário

In [3]:
# Número do arquivo do voluntário, alterar apenas o número

''' Alterar o número do sujeito analisado'''

voluntary = 15

In [4]:
voluntary = str(voluntary)
v3d_filename = caminho + 'F_v3d_Subject_' + voluntary + '.mat'

# Escolher a chave

In [5]:
# Reading MoVi files as a Python dictionary
sample = mat2dict(v3d_filename)
sample.keys()

dict_keys(['id', 'subject', 'move'])

In [6]:
# Printing the dictionary keys in a tree style
pretty_dict(sample, print_type=False, indent=1)

	Key: id
	Key: subject
		Key: id
		Key: sex
		Key: handedness
		Key: height
		Key: mass
		Key: age
	Key: move
		Key: description
		Key: markerName
		Key: markerType
		Key: markerDescription
		Key: markerSide
		Key: markerMatch
		Key: segmentName
		Key: segmentType
		Key: segmentDescription
		Key: segmentSide
		Key: segmentMatch
		Key: segmentParent
		Key: physicalMarkers
		Key: virtualMarkers
		Key: lcData
		Key: flags30
		Key: flags120
		Key: motions_list
		Key: markerGaps
		Key: virtualMarkerParent
		Key: markerLocation
		Key: virtualMarkerLocation
		Key: jointsAffine_v3d
		Key: jointsTranslation_v3d
		Key: jointsExpMaps_v3d
		Key: jointsGaps_v3d


In [7]:
# First move sample
move = sample['move']
pretty_dict(move)

Key: description
Key: markerName
Key: markerType
Key: markerDescription
Key: markerSide
Key: markerMatch
Key: segmentName
Key: segmentType
Key: segmentDescription
Key: segmentSide
Key: segmentMatch
Key: segmentParent
Key: physicalMarkers
Key: virtualMarkers
Key: lcData
Key: flags30
Key: flags120
Key: motions_list
Key: markerGaps
Key: virtualMarkerParent
Key: markerLocation
Key: virtualMarkerLocation
Key: jointsAffine_v3d
Key: jointsTranslation_v3d
Key: jointsExpMaps_v3d
Key: jointsGaps_v3d


In [8]:
move =dict2ntuple(move)
# print(move.virtualMarkerLocation)

--> Carregue os dados do código anterior que calcula o centro de massa do corpo todo para todos os frames.

In [9]:
array = np.load('Whole body CoM data.npy')

--> Criar o dataframe

In [10]:
Data = array
name_description = move.motions_list
data_complete = pd.DataFrame(Data,
                           columns = ['X', 'Y'])
data_complete

,X,Y
0,18.929139,986.508717
1,18.938636,986.511618
2,18.950765,986.516038
3,19.017671,986.494681
4,19.000997,986.499087
...,...,...
17841,46.894255,939.087568
17842,46.896500,939.094325
17843,46.874120,939.095647
17844,46.877768,939.117538


# Separando apenas os dados do movimento de caminhar de um lado ao outro

In [11]:
video_path = r'C:\\Users\\gusta\\OneDrive\\Documentos\\Users\\Gustavo Monteiro\\Documents\\GitHub\\MoVi-Toolbox\\MoCap\\'

video_F = 'F'

In [12]:
# Frequencia dos dados

video_name = video_path + video_F + '_PG1_Subject_' + voluntary + '_L.avi'

freq_MoCap = 120
n_frames, fps = count_frames(video_name)

# Escolher o tempo de início aproximado para o movimento de andar de um lado ao outro

In [13]:
print('Utilizar o vídeo com o seguinte nome: ', video_F + '_PG1_Subject_' + voluntary + '_L.avi' )

Utilizar o vídeo com o seguinte nome:  F_PG1_Subject_15_L.avi


--> Estabeleça o minuto e segundos exatos do início do trecho do vídeo que você busca.

In [14]:
# alterar o minuto do vídeo para o movimento
min_init_video = 1
# alterar o segundo do vídeo para o movimento 
seg_init_video = 55

aprox_init_video = int(((min_init_video * 60) + seg_init_video) * fps)

--> Estabeleça manualmente o trecho do vídeo que deseja coletar. Aperte a tecla D repetidas vezes para os frames do vídeo irem para frente e A para recuarem. 

--> Após chegar no frame de início que você deseja, aperte Enter e após chegar no frame final, aperte Enter novamente.

In [19]:
start_end = defineStartEnd(video_name, n_frames, aprox_init_video)

[3466, 0]
[3466, 3504]


--> Rode o bloco de código seguinte para converter para a frequência de dados do sistema mocap (120 Hz) e cria o arquivo com esses dados separados.

In [23]:
# frame de início e final do movimento
frame_start = start_end[0]
frame_end = start_end[1]

# converter o frame do vídeo do início do movimento para o dado de Mocap correspondente 
temp_start = frame_start / fps
MOCAP_start = int(temp_start * freq_MoCap)

# converter o frame do vídeo do final do movimento para o dado de Mocap correspondente 
temp_end = frame_end / fps
MOCAP_end = int(temp_end * freq_MoCap)

# criando os dados apenas com as informações 
data_gait = data_complete.iloc[MOCAP_start:MOCAP_end + 1,:]
len(data_gait)
data_gait

,X,Y
13864,977.471806,912.392612
13865,968.700176,911.827323
13866,959.899099,911.299103
13867,950.833143,911.069982
13868,941.732116,911.147434
...,...,...
14012,-385.008500,912.643580
14013,-394.356797,913.402616
14014,-403.671890,914.516424
14015,-413.226645,915.680980


In [24]:
np.save('MoCap CoM gait', data_gait)